#### Name    - Roll no.
#### Ajith      - E20003
#### James   - E20013
#### Swastik - E20036

### We have developed 3 recommendation model<br>

<ol>
<li> A simple recommendation model based on movie scores. Novie scores are obtained by a weighted rating formula. We can get a list of top x movies using this model. <br><br>
<li> A sub genre and timeframe based model. It is built by clustering movies based on genres and filtering based on preferred time period. <br>Models 1 and 2 are implemented together<br><br>
<li> A recommendation model based on user's recently liked movies. It is built by using movie - movie cosine similarity matrix + user's recent viewed(& liked) history <br><br>
    
* We have used the smaller version of the data due to space constraints

# IMPORTING PACKAGES

In [1]:
import pandas as pd
import numpy as np

## READING DATA

In [2]:
movies = pd.read_csv("D:\\Praxis\\ML\\data\\ml-latest-small\\movies.csv")

In [3]:
movies.sample(2)

,movieId,title,genres
4710,7027,Silverado (1985),Action|Western
1210,1609,187 (One Eight Seven) (1997),Drama|Thriller


In [4]:
### Splitting genre column into separate boolean genre columns

In [5]:
genres_list = []
for genre in movies['genres']:
    genre_splitted = genre.split('|')
    for item in genre_splitted:
        if item not in genres_list:
            genres_list.append(item)

#movies['genres'].split('|')
for genre in genres_list:
    movies[genre] = 0
movies.sample(2)

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
4013,5670,Comedian (2002),Comedy|Documentary,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2794,3736,"Ace in the Hole (Big Carnival, The) (1951)",Drama,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# filling 1 for genre being present

for index, row in movies.iterrows():
    for current_genre in genres_list:
        if current_genre in row['genres']:
            movies.loc[index,current_genre] = 1

In [7]:
movies.sample(2)

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
5294,8771,Sherlock Holmes: Terror by Night (1946),Crime|Mystery|Thriller,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5060,7932,Dark Days (2000),Documentary,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


## Model 1 - CLUSTERING BASED ON GENRES

* We have used Agglomerative clustering with Cosine similarity as the metric. 
* We are setting number of clusters to be created as 8. Tried with different cluster numbers. Found 8 to be the appropriate choice.
* We look at the genres in each cluster with the most occurences. Based on the frequency, we give it a generalised category name - Eg: Romcom
* After this, we want to provide the choice of category. We also want to provide the choice of timeframe, for eg: 90s
* To do this, we extract the year_released from the movie title column and filter based on his choice.
* He can also choose that he has no preference.

In [9]:
# Slicing only the genre columns

movies_sub = movies.iloc[:,3:]
movies_sub.sample(2)

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
3154,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1753,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
# importing packages
import sklearn
from sklearn.cluster import AgglomerativeClustering

In [11]:
hclust = AgglomerativeClustering(n_clusters = 8,affinity = 'cosine', linkage = 'complete')

In [12]:
hclust.fit(movies_sub)

AgglomerativeClustering(affinity='cosine', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='complete', memory=None, n_clusters=8,
                        pooling_func='deprecated')

In [13]:
movies['groups'] = hclust.labels_

### Finding genres with highest frequency within each group

In [14]:
genre_list = movies[movies['groups'] == 0]['genres'].value_counts()
genre_list[0:10]


# Horror-Thriller

Horror                  167
Horror|Thriller         135
Crime|Drama             134
Crime|Drama|Thriller    125
Drama|War               114
Comedy|Crime            101
Thriller                 84
Children|Comedy          74
Horror|Sci-Fi            53
Crime|Thriller           45
Name: genres, dtype: int64

In [15]:
genre_list = movies[movies['groups'] == 1]['genres'].value_counts()
genre_list[0:10]

# Romcom

Comedy                    946
Comedy|Drama              435
Comedy|Romance            363
Comedy|Drama|Romance      276
Comedy|Horror              69
Adventure|Comedy           36
Comedy|Fantasy             33
Comedy|Documentary         31
Animation|Comedy           30
Comedy|Fantasy|Romance     24
Name: genres, dtype: int64

In [16]:
genre_list = movies[movies['groups'] == 2]['genres'].value_counts()
genre_list[0:10]

# Action

Action|Comedy                  92
Action|Crime|Thriller          66
Action|Drama                   62
Action|Crime|Drama|Thriller    61
Action                         60
Action|Thriller                60
Action|Sci-Fi|Thriller         43
Action|Drama|Thriller          43
Action|Adventure|Fantasy       41
Action|Adventure|Thriller      40
Name: genres, dtype: int64

In [17]:
genre_list = movies[movies['groups'] == 3]['genres'].value_counts()
genre_list[0:10]

#Drama - movie with lots of suspense and action.

Drama                            1053
Drama|Thriller                    168
Action|Crime|Drama                 50
Drama|Musical                      44
Action|Drama|War                   41
Drama|Horror|Thriller              40
Drama|Sci-Fi                       30
Drama|Horror                       26
Drama|Fantasy                      24
Drama|Horror|Mystery|Thriller      20
Name: genres, dtype: int64

In [18]:
genre_list = movies[movies['groups'] == 4]['genres'].value_counts()
genre_list[0:10]

# Documentary

Documentary                         339
Documentary|Drama                    13
Documentary|War                       9
Documentary|IMAX                      6
Crime|Documentary                     4
Comedy|Documentary|Drama              2
Documentary|Musical|IMAX              1
Documentary|Drama|Thriller            1
Comedy|Documentary|Drama|Romance      1
Crime|Documentary|War                 1
Name: genres, dtype: int64

In [19]:
genre_list = movies[movies['groups'] == 5]['genres'].value_counts()
genre_list[0:10]

# Animation

Animation                          30
Action|Animation|Crime              3
Animation|Documentary               2
Action|Animation|Children|Crime     2
Animation|Crime|Drama               1
Action|Animation|Crime|Drama        1
Name: genres, dtype: int64

In [20]:
genre_list = movies[movies['groups'] == 6]['genres'].value_counts()
genre_list[0:10]

# Children

Action|Adventure|Sci-Fi                66
Adventure|Drama                        41
Adventure|Animation|Children|Comedy    31
Adventure|Children                     23
Adventure|Animation|Children           16
Adventure|Sci-Fi                       15
Adventure|Animation|Comedy             13
Adventure|Comedy|Sci-Fi                13
Adventure|Comedy|Drama                 12
Adventure                              12
Name: genres, dtype: int64

In [47]:
genre_list = movies[movies['groups'] == 7]['genres'].value_counts()
genre_list[0:10]

# Romance

Drama|Romance                     349
Romance                            21
Drama|Musical|Romance              21
Drama|Fantasy|Romance              18
Adventure|Drama|Romance            15
Adventure|Comedy|Romance           13
Drama|Romance|Thriller             11
Drama|Mystery|Romance               9
Drama|Mystery|Romance|Thriller      9
Crime|Drama|Romance|Thriller        9
Name: genres, dtype: int64

In [21]:
# combining groups 5 and 6 since they both have the same type of movies and group 6 has considerably less number of movies

movies.loc[movies['groups']==6,'groups']= 5
movies.loc[movies['groups']==7,'groups']= 6

In [22]:
movies.shape

(9742, 24)

## Model 2 - RECOMMENDATION BASED ON MOVIE RATINGS

* We have used a weighted rating formula to find the movie score. (Ref - https://www.datacamp.com/community/tutorials/recommender-systems-python)
* We can see the highest rated movies after doing a descending sort on the movie score.

In [23]:
ratings = pd.read_csv(("D:\\Praxis\\ML\\data\\ml-latest-small\\ratings.csv"))

In [24]:
data = pd.merge(ratings,movies, how = 'left', on = 'movieId')

In [25]:
data.sample(2)

,userId,movieId,rating,timestamp,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),groups
46326,305,56156,4.5,1460137045,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,2
9501,64,595,3.5,1161520447,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,0,1,0,0,0,0


In [26]:
df = data.groupby('movieId',as_index=False)['rating'].mean()

In [27]:
df['votes'] = data.groupby('movieId',as_index=False).count()['rating']

In [28]:
df.sample(5)

,movieId,rating,votes
2105,2800,2.500000,1
2336,3096,5.000000,1
7001,68358,3.864407,59
7891,95182,3.000000,2
531,626,5.000000,1


In [29]:
def recommender_func(data,min_votes):
  nw_data = data.copy()

  nw_data = nw_data.loc[nw_data['votes'] >= min_votes]
  denom = nw_data['votes'] + min_votes
  WR = ((nw_data['votes']/denom) * nw_data['rating']) + ((min_votes/denom) * np.mean(nw_data['rating']))

  nw_data['scores'] = WR
  return nw_data


In [30]:
top_movies = recommender_func(df,10)
movie_df = data.groupby('movieId',as_index=False).min()[['movieId','title','genres','groups']]

top_movies = top_movies.merge(movie_df, how = 'inner', on = 'movieId').sort_values('scores',ascending=False)
top_movies.head()

,movieId,rating,votes,scores,title,genres,groups
158,318,4.429022,317,4.398534,"Shawshank Redemption, The (1994)",Crime|Drama,0
346,858,4.289062,192,4.246636,"Godfather, The (1972)",Crime|Drama,0
1055,2959,4.272936,218,4.236055,Fight Club (1999),Action|Crime|Drama|Thriller,2
131,260,4.231076,251,4.200462,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,5
488,1221,4.259690,129,4.200148,"Godfather: Part II, The (1974)",Crime|Drama,0


In [31]:
# adding category names to the top movies dataframe

top_movies.loc[top_movies['groups'] == 6,'Category'] = 'Romance'
top_movies.loc[top_movies['groups'] == 5,'Category'] = 'Animation'
top_movies.loc[top_movies['groups'] == 4,'Category'] = 'Documentary'
top_movies.loc[top_movies['groups'] == 3,'Category'] = 'Drama'
top_movies.loc[top_movies['groups'] == 2,'Category'] = 'Action-Thriller'
top_movies.loc[top_movies['groups'] == 1,'Category'] = 'Romcom'
top_movies.loc[top_movies['groups'] == 0,'Category'] = 'Horror Thriller'

In [32]:
# Extracting year_released from movie title

years = []

for title in top_movies.title:
    if title.find('(18') != -1:
        x = title.find('(18')
        years.append(int(title[x+1:x+5]))
    
    elif title.find('(19') != -1:
        x = title.find('(19')
        years.append(int(title[x+1:x+5]))
        
    elif title.find('(20') != -1:
        x = title.find('(20')
        years.append(int(title[x+1:x+5]))
    
    else:
        years.append(9999)

In [33]:
top_movies['year_released'] = years

### Filtering based on decades, say 70s, 80s, 90s

In [34]:
top_movies = top_movies.sort_values(by = 'scores',ascending = False)
top_movies.head(2)

,movieId,rating,votes,scores,title,genres,groups,Category,year_released
158,318,4.429022,317,4.398534,"Shawshank Redemption, The (1994)",Crime|Drama,0,Horror Thriller,1994
346,858,4.289062,192,4.246636,"Godfather, The (1972)",Crime|Drama,0,Horror Thriller,1972


In [37]:
# movie titles have 'the' displayed at the end. Taking it to the start using a simple function

def the_fn(s):
    if (", The" in s):
        s = s.replace(", The","")
        s = "The " + s
    return s

x = the_fn("Ajith, The")
x

In [38]:
xd = ([the_fn(x) for x in top_movies['title']])

In [39]:
top_movies.drop(['title'], axis=1)
top_movies['title'] = xd

### Suggesting movies based on timeline and category of their choice

In [66]:
top_movies = top_movies.sort_values(by = 'scores',ascending = False)

print("\nCategories") #option for multiple ctegories next
print("-"*10)
print("1. Romance\t2. Animation\t3.Documentary\n4.Drama\t5.Action-Thriller\t6.Romcom\n7.Horror Thriller\t8.No Preference\n")
category = input('What category do you want to check out?')
print("\n")
year = int(input("Movies from what time do you like? For example 90s. You can also say no preference... "))
print("\n")

if (year < 20):
    starting_year = year + 2000
else:
    starting_year = year + 1900

ending_year = starting_year + 9

x = (top_movies.loc[(top_movies['Category']==category) & (top_movies['year_released'] >= starting_year) & (top_movies['year_released'] <= ending_year)]['title'].head(10).values)
rec_movies = pd.DataFrame(x)
rec_movies.columns = ['Title']
rec_movies


Categories
----------
1. Romance	2. Animation	3.Documentary
4.Drama	5.Action-Thriller	6.Romcom
7.Horror Thriller	8.No Preference

What category do you want to check out?Romance


Movies from what time do you like? For example 90s. You can also say no preference... 00




,Title
0,The Lives of Others (Das leben der Anderen) (2...
1,"Beautiful Mind, A (2001)"
2,In the Mood For Love (Fa yeung nin wa) (2000)
3,Cinderella Man (2005)
4,"Crouching Tiger, Hidden Dragon (Wo hu cang lon..."
5,And Your Mother Too (Y tu mamá también) (2001)
6,Talk to Her (Hable con Ella) (2002)
7,Big Fish (2003)
8,The Girl Who Leapt Through Time (Toki o kakeru...
9,The Illusionist (2006)


,Title
0,The Lives of Others (Das leben der Anderen) (2...
1,"Beautiful Mind, A (2001)"
2,In the Mood For Love (Fa yeung nin wa) (2000)
3,Cinderella Man (2005)
4,"Crouching Tiger, Hidden Dragon (Wo hu cang lon..."
5,And Your Mother Too (Y tu mamá también) (2001)
6,Talk to Her (Hable con Ella) (2002)
7,Big Fish (2003)
8,The Girl Who Leapt Through Time (Toki o kakeru...
9,The Illusionist (2006)


## Model 3 - RECOMMENDATION BASED ON MOVIE - MOVIE SIMILARITY

* We have created the cosine similarity matrix for movies. This will give us the most similar movies to a specific movie.
* Using 'ratings.csv', for a particular user, we filter to get recently watched(using a descending sort on timestamp) which he liked(based on rating).
* For those movies, we will find 5 most similar movies using cosine sim matrix.

In [41]:
from sklearn.metrics.pairwise import cosine_similarity
cos_movies = pd.DataFrame(cosine_similarity(movies_sub))
cos_movies.index = movies['title']
cos_movies.columns = movies['title']

### Using recent movies Mr.X has watched and liked, suggesting similar movies to that

In [42]:
def recent_movies(id, top = 5):
    ratings1 = ratings.copy()
    ratings1.sort_values(by = ['userId','timestamp','rating'], ascending=[True,False,False], inplace=True)

    ratings1 = ratings1.loc[ratings1['userId'] == id]
    max_rating =  ratings1['rating'].max()
    
    if max_rating <= 4.0:
        ratings1 = ratings1.loc[ratings1['rating'] == max_rating]
        return (ratings1.head(top)['movieId']).tolist()
    else:
        ratings1 = ratings1.loc[ratings1['rating'] >= 4.0]
        return (ratings1.head(top)['movieId']).tolist()

In [43]:
recent_liked_movies_ids = recent_movies(1)

In [44]:
recent_liked_movies_names = []
for mid in recent_liked_movies_ids:
    recent_liked_movies_names.append(movies.loc[movies['movieId'] == mid, 'title'].values[0])
recent_liked_movies_names

['20 Dates (1998)',
 'Back to the Future Part III (1990)',
 '¡Three Amigos! (1986)',
 'Tombstone (1993)',
 'Canadian Bacon (1995)']

In [55]:
print("Your recommendations based on recent watch history are...\n")
for movie in recent_liked_movies_names:
    print("\n>>>>",movie)
    very_similar_movies = cos_movies.loc[movie,:].sort_values(ascending=False).head(5) 
    very_similar_movies_indices = very_similar_movies.index
    print(list(very_similar_movies_indices))

Your recommendations based on recent watch history are...


>>>> 20 Dates (1998)
['Just Friends (2005)', 'When in Rome (2010)', "Say It Isn't So (2001)", 'Leap Year (2010)', 'The Girls (1961)']

>>>> Back to the Future Part III (1990)
['Back to the Future Part III (1990)', 'Back to the Future Part II (1989)', 'Our Man Flint (1965)', 'Almost Heroes (1998)', 'Paul (2011)']

>>>> ¡Three Amigos! (1986)
['City Slickers (1991)', 'Scalphunters, The (1968)', 'Trinity and Sartana Are Coming (1972)', 'Texas - Doc Snyder hält die Welt in Atem (1993)', 'Blazing Saddles (1974)']

>>>> Tombstone (1993)
["Once Upon a Time in the West (C'era una volta il West) (1968)", 'Django Unchained (2012)', 'Tombstone (1993)', 'Jonah Hex (2010)', 'Treasure of the Sierra Madre, The (1948)']

>>>> Canadian Bacon (1995)
['Guy X (2005)', 'Canadian Bacon (1995)', 'Whiskey Tango Foxtrot (2016)', '1941 (1979)', 'And the Ship Sails On (E la nave va) (1983)']
